In [1]:
# Importar bibliotecas necessárias
import pandas as pd
import re
import datetime as date
from conexao_bancodedados import *

In [2]:
# Definir o caminho do arquivo CSV
csv_file = r'C:\Users\Lucas\OneDrive\Documentos\projetos_python\Jornada_financas_pessoais\data\Controle de Ações e FII - Yasmin - FII.csv'

# Pega o nome do acionista que está entre os "- -"
nome_acionista = re.search(r'- (.*) -', csv_file).group()


# Ler o arquivo CSV delimitado por vírgula
df = pd.read_csv(csv_file, sep=',')

df.head()

,DATA,PAPEL,OP,QTD,PREÇO,CUS.T.,TOTAL OP,RAT.,V. LIQUIDO,ESTOQUE,PMEDIO,GAN/PER,IR MÊS,MÊS REF.,VENDAS/MÊS
0,26/10/2021,LVBI11,C,2,"97,78","0,19","195,56","0,05","-195,61",2,"97,805",NaN,NaN,10/2021,"0,00"
1,26/10/2021,RBED11,C,2,"132,10","0,19","264,20","0,07","-264,27",2,"132,135",NaN,NaN,10/2021,"0,00"
2,26/10/2021,TRXF11,C,2,"101,21","0,19","202,42","0,05","-202,47",2,"101,235",NaN,NaN,10/2021,"0,00"
3,08/11/2021,XPML11,C,6,"96,47","0,16","578,82","0,16","-578,98",6,"96,497",NaN,NaN,11/2021,"0,00"
4,26/11/2021,HGRU11,C,1,"102,65","0,19","102,65","0,02","-102,67",1,"102,670",NaN,NaN,11/2021,"0,00"


In [3]:
# Renomear as colunas do DataFrame para corresponder aos nomes das colunas no MySQL
df.rename({
    'DATA':'data',
    'PAPEL':'papel',
    'OP':'op',
    'QTD':'qtd',
    'PREÇO':'preco',
    'CUS.T.':'custot',
    'TOTAL OP':'totlop',
    'RAT.':'rat',
    'V. LIQUIDO':'vliquido',
    'ESTOQUE':'estoque',
    'PMEDIO':'pmedio',
    'GAN/PER':'gan_per',
    'IR MÊS':'ir_mes',
    'MÊS REF.':'mes_ref',
    'VENDAS/MÊS':'vendas_mes'
}, axis=1, inplace=True)

# Adiciona coluna com nome acionista
df['acionista'] = nome_acionista

# Remove caracteres especiais
df['acionista'] = df['acionista'].str.replace('-','')

# Adiciona coluna de data_insercao
df['data_insercao'] = date.datetime.today()

# Remove coluna réplica
df.drop(columns='vendas_mes', axis=1)

# Exibe Dataframe
df.head()


,data,papel,op,qtd,preco,custot,totlop,rat,vliquido,estoque,pmedio,gan_per,ir_mes,mes_ref,vendas_mes,acionista,data_insercao
0,26/10/2021,LVBI11,C,2,"97,78","0,19","195,56","0,05","-195,61",2,"97,805",NaN,NaN,10/2021,"0,00",Yasmin,2024-11-18 18:10:52.458444
1,26/10/2021,RBED11,C,2,"132,10","0,19","264,20","0,07","-264,27",2,"132,135",NaN,NaN,10/2021,"0,00",Yasmin,2024-11-18 18:10:52.458444
2,26/10/2021,TRXF11,C,2,"101,21","0,19","202,42","0,05","-202,47",2,"101,235",NaN,NaN,10/2021,"0,00",Yasmin,2024-11-18 18:10:52.458444
3,08/11/2021,XPML11,C,6,"96,47","0,16","578,82","0,16","-578,98",6,"96,497",NaN,NaN,11/2021,"0,00",Yasmin,2024-11-18 18:10:52.458444
4,26/11/2021,HGRU11,C,1,"102,65","0,19","102,65","0,02","-102,67",1,"102,670",NaN,NaN,11/2021,"0,00",Yasmin,2024-11-18 18:10:52.458444


In [4]:
# Truncar os dados da tabela antes de carregar
truncate_table('bronze','controle_acoes_fiis')

In [ ]:
# Conexão com o banco de dados na camada "bronze":
engine = conexao_mysql('bronze')

df.to_sql(name='controle_acoes_fiis', con=engine, if_exists='append', index=False)


-1